In [37]:
import pandas as pd
import jdatetime
from datetime import timedelta

In [38]:
df = pd.read_csv('Project3_Dataset.csv')

df.head(2)

,job_id,date,job_title,experience_level,city,remote_option,company_size,gender,skills_required,salary_million_tmn
0,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,1403-11-22,DevOps Engineer,Junior,Tehran,False,Medium,Male,"['CI/CD', 'Linux Administration', 'Monitoring ...",22.49
1,c12e7ca2-cfae-4eeb-9091-09d5c6c2d655,1403-02-27,Data Analyst,Expert,Isfahan,False,Small,Male,"['Excel', 'Presentation', 'Data Analysis']",64.01


In [39]:

# ایجاد بازه تاریخ
start_date = jdatetime.date(1403, 1, 1)
end_date = jdatetime.date(1404, 12, 29)
date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# ساخت دیتافریم
dim_date = pd.DataFrame({
    'date': [str(d) for d in date_list],
    'date_id': range(1, len(date_list) + 1),
    'year': [d.year for d in date_list],
    'month': [d.month for d in date_list],
    'day': [d.day for d in date_list],
    'month_name': [d.strftime('%B') for d in date_list],  # نام ماه به فارسی
    'quarter': [(d.month - 1) // 3 + 1 for d in date_list],  # فصل
    'daypulse': [d.weekday() for d in date_list]
})



In [40]:
# جایگزینی مقادیر ستون date در df با مقدار date_id از dim_date بر اساس ستون date
df['date'] = df['date'].map(dim_date.set_index('date')['date_id']).combine_first(df['date'])


In [41]:
# پیدا کردن مقادیری که نوع داده آنها در ستون date متفاوت است
df[df['date'].apply(lambda x: not isinstance(x, (int, float)))].date


16       1403-15-01
67       1403-02-40
328      1404-03-54
562      1704-01-14
3944     1403-18-23
4080    1403-03-325
4330     1403-16-14
4814     1403-02-47
5152     1403-18-18
5546     1404-13-42
5716     1403-00-24
6041     1803-09-58
6536     1403-22-21
Name: date, dtype: object

In [42]:
# بر اساس ایندکس‌های داده شده، مقدار ستون date را در df اصلاح می‌کنیم
date_corrections = {
    16: '1403-12-01',
    67: '1403-02-04',
    328: '1404-03-04',
    562: '1404-01-14',
    3944: '1403-08-23',
    4080: '1403-03-25',
    4330: '1403-06-14',
    4814: '1403-02-07',
    5152: '1403-08-18',
    5546: '1404-03-02',
    5716: '1403-01-24',
    6041: '1403-09-08',
    6536: '1403-12-21'
}

for idx, new_date in date_corrections.items():
    if idx in df.index:
        df.at[idx, 'date'] = new_date

df['date'] = df['date'].map(dim_date.set_index('date')['date_id']).combine_first(df['date'])
df.rename(columns={'date': 'date_id'}, inplace=True)
df['date_id'] = df['date_id'].astype(int)


In [43]:
# برای هر سال و ماه، لیست روزها را چاپ می‌کنیم
from collections import defaultdict

# فرض می‌کنیم ستون date_id در df وجود دارد و معادل تاریخ میلادی است
# اگر نه، از ستون date (که به صورت جلالی است) استفاده می‌کنیم
# فرمت تاریخ: YYYY-MM-DD

days_by_year_month = defaultdict(list)

for date_str in df['date_id']:
    # اگر مقدار عددی است (date_id)، باید آن را به رشته تبدیل کنیم یا از dim_date برگردانیم
    if isinstance(date_str, (int, float)):
        # پیدا کردن تاریخ جلالی معادل date_id
        jalali_date = dim_date.loc[dim_date['date_id'] == date_str, 'date']
        if not jalali_date.empty:
            date_str = jalali_date.values[0]
        else:
            continue  # اگر پیدا نشد، رد می‌کنیم

    # حالا date_str باید به صورت 'YYYY-MM-DD' باشد
    try:
        year, month, day = date_str.split('-')
        days_by_year_month[(year, month)].append(day)
    except Exception:
        continue

# چاپ لیست روزها برای هر سال و ماه
for (year, month), days in sorted(days_by_year_month.items()):
    print(f"Year: {year}, Month: {month}, Days: {sorted(set(days), key=lambda x: int(x))}")




Year: 1403, Month: 01, Days: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
Year: 1403, Month: 02, Days: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
Year: 1403, Month: 03, Days: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
Year: 1403, Month: 04, Days: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
Year: 1403, Month: 05, Days: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18'

In [44]:
df.city.unique()

array(['Tehran', 'Isfahan', 'Kashan', 'Kerman', 'Mashhad', 'Zanjan',
       'Shiraz', 'Sari', 'Khorramabad', 'Rasht', 'Ahvaz', 'Babol',
       'Qazvin', 'Sabzevar', 'Arak', 'Karaj', 'Ardabil', 'Qom', 'Zahedan',
       'Neyshabur', 'Tabriz', 'Urmia', 'Amol', 'Gorgan', 'Bushehr',
       'Dezful', 'Semnan', 'Hamadan', 'Bojnurd', 'Shahrekord', 'Yazd',
       'Yasuj', 'Birjand', 'Abadan', 'Sanandaj', 'Kermanshah', 'Ilam',
       'Bandar Abbas', 'Mashhad*-', 'Sanandaj*/', 'Tehran*', 'Urmia/',
       'Karaj+', 'Gorgan*/', 'Tehran9+', 'Shiraz+-'], dtype=object)

In [45]:
import re

def clean_city(city):
    # Remove any character that is not a letter (Persian or English) or space
    return re.sub(r'[^a-zA-Zآ-ی\s]', '', str(city))

df['city'] = df['city'].apply(clean_city)


In [46]:
df.city.unique()

array(['Tehran', 'Isfahan', 'Kashan', 'Kerman', 'Mashhad', 'Zanjan',
       'Shiraz', 'Sari', 'Khorramabad', 'Rasht', 'Ahvaz', 'Babol',
       'Qazvin', 'Sabzevar', 'Arak', 'Karaj', 'Ardabil', 'Qom', 'Zahedan',
       'Neyshabur', 'Tabriz', 'Urmia', 'Amol', 'Gorgan', 'Bushehr',
       'Dezful', 'Semnan', 'Hamadan', 'Bojnurd', 'Shahrekord', 'Yazd',
       'Yasuj', 'Birjand', 'Abadan', 'Sanandaj', 'Kermanshah', 'Ilam',
       'Bandar Abbas'], dtype=object)

In [47]:
df.experience_level.unique()

array(['Junior', 'Expert', 'Mid-level', 'Senior', 'Intern', 'Lead',
       'Mid-level/*', 'Mid-level+/*'], dtype=object)

In [48]:
def clean_experience_level(level):
    # Remove any character that is not a letter (Persian or English) or space
    return re.sub(r'[^a-zA-Zآ-ی\s]', '', str(level))

df['experience_level'] = df['experience_level'].apply(clean_experience_level)


In [49]:
df.experience_level.unique()

array(['Junior', 'Expert', 'Midlevel', 'Senior', 'Intern', 'Lead'],
      dtype=object)

In [50]:
# Function to parse and separate the list string
def parse_skills(skill_string):
    # Remove outer brackets and extract quoted strings
    cleaned = skill_string.strip("[]")
    elements = re.findall(r"'([^']*)'", cleaned)
    result = []
    for element in elements:
        if '(' in element:
            # Split into main term and parenthetical content
            main, rest = element.split(' (')
            rest = rest.rstrip(')')
            # Add main term and split parenthetical content
            result.append(main.strip())
            result.extend(x.strip() for x in rest.split(','))
        else:
            result.append(element.strip())
    return result

df['skills_required'] = df['skills_required'].apply(parse_skills)

In [51]:
df.skills_required

0       [CI/CD, Linux Administration, Monitoring Tools...
1                    [Excel, Presentation, Data Analysis]
2                          [Excel, Presentation, PowerBI]
3                                            [AI, Python]
4                    [Tableau, SQL, Presentation, Python]
                              ...                        
7195    [Machine Learning, Python, Deep Learning, Math...
7196                                       [iOS, Android]
7197    [Object-Oriented Programming, API Development,...
7198    [Excel, Tableau, Presentation, PowerBI, Machin...
7199          [Network Configuration, LAN/WAN Management]
Name: skills_required, Length: 7200, dtype: object

In [52]:
df.skills_required.value_counts()

skills_required
[Python, Spark]                                            17
[SQL, Python]                                              17
[Spark, Python]                                            16
[Python, SQL]                                              15
[C++, Unity]                                               14
                                                           ..
[PowerBI, Presentation, Excel, Python]                      1
[LAN/WAN Management, Monitoring Tools, Troubleshooting]     1
[Presentation, Data Analysis, PowerBI, Excel]               1
[SQL, Spark, Big Data, Python]                              1
[NoSQL, Spark, Hadoop]                                      1
Name: count, Length: 4942, dtype: int64

In [55]:
# Create a DataFrame where each row is exploded by 'skills_required' so each skill gets its own row
df_exploded = df.explode('skills_required').reset_index(drop=True)
df_exploded.head(3)

,job_id,date_id,job_title,experience_level,city,remote_option,company_size,gender,skills_required,salary_million_tmn
0,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,328,DevOps Engineer,Junior,Tehran,False,Medium,Male,CI/CD,22.49
1,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,328,DevOps Engineer,Junior,Tehran,False,Medium,Male,Linux Administration,22.49
2,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,328,DevOps Engineer,Junior,Tehran,False,Medium,Male,Monitoring Tools,22.49


In [53]:
# Create a DataFrame with unique skills, sorted alphabetically, and assign skill_id starting from 1
unique_skills = sorted(set(skill for skills in df['skills_required'] for skill in skills))
dim_skill = pd.DataFrame({
    'skill_id': range(1, len(unique_skills) + 1),
    'skill': unique_skills
})
print('dim_skill shape:', dim_skill.shape)
dim_skill.head(3)

dim_skill shape: (97, 2)


,skill_id,skill
0,1,AI
1,2,API Development
2,3,AWS


In [54]:
# Create a bridge table between df and dim_skill with columns: job_id and skill_id

# First, create a mapping from skill name to skill_id
skill_to_id = dict(zip(dim_skill['skill'], dim_skill['skill_id']))

# For each row in df, create (job_id, skill_id) pairs
bridge_rows = []
for idx, row in df.iterrows():
    job_id = row['job_id']
    for skill in row['skills_required']:
        skill_id = skill_to_id.get(skill)
        if skill_id is not None:
            bridge_rows.append({'job_id': job_id, 'skill_id': skill_id})

job_bridge_skill = pd.DataFrame(bridge_rows)
print('df_job_skill shape:', df_job_skill.shape)
df_job_skill.head(3)


df_job_skill shape: (30076, 2)


,job_id,skill_id
0,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,14
1,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,49
2,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,55


In [4]:
import pandas as pd

dim_inflation = pd.DataFrame({
    'Inflation_Rate (%)': [
        33.2, 33.6, 34.5, 35.5, 35.0, 34.0, 33.6, 33.1,
        33.0, 33.5, 34.0, 34.5, 35.0, 38.9, 38.7, 39.0
    ],
    'USD_to_IRR (Free Market Avg)': [
        550000, 580000, 600000, 620000, 650000, 680000, 700000, 750000,
        800000, 820000, 850000, 880000, 900000, 920000, 930000, 940000
    ],
    'date': [
        '1403-01', '1403-02', '1403-03', '1403-04', '1403-05', '1403-06', '1403-07', '1403-08',
        '1403-09', '1403-10', '1403-11', '1403-12', '1404-01', '1404-02', '1404-03', '1404-04'
    ],
    'date_id': [
        '1', '32', '63', '94', '125', '156', '187', '217',
        '247', '277', '307', '337', '367', '398', '429', '460'
    ]
})


## Save CSV file

In [ ]:

df.to_csv('cleand_.csv', index=False)

In [69]:
dim_date.to_csv('dim_date.csv', index=False)

In [36]:
job_bridge_skill.to_csv('job_bridge_skill.csv', index=False)

In [34]:
dim_skill.to_csv('dim_skill.csv', index=False)

In [58]:
df_exploded.to_csv('job_skill_exploded.csv', index=False)

In [5]:
dim_inflation.to_csv('dim_inflation.csv', index=False)